# ETL Pipeline Preparation

Workspace to help develop ETL pipeline to be used in process_data.py.

Format for process_data.py is:
python process_data.py disaster_messages.csv disaster_categories.csv DisasterResponse.db

### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [112]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine

In [2]:
# load datasets
data_directory = "./data/"
message_file = "messages.csv"
categories_file = "categories.csv"
table_name = "fergus."

messages = pd.read_csv(data_directory + message_file)
categories = pd.read_csv(data_directory + categories_file)


In [23]:
print(messages.shape)
messages.head()

(26248, 4)


,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [108]:
print(categories.shape)
categories.head()

(26180, 2)


,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [29]:
messages[messages['id']==25512]

,id,message,original,genre
22027,25512,The flood situation has worsened with nearly 3...,NaN,news
22028,25512,The flood situation has worsened with nearly 3...,NaN,news


In [26]:
messages.iloc[21360,1]

'Shelter materials (thick polyesters) are being distributed to 18,000 households.'

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [43]:
messages.drop_duplicates(subset='id', keep='first', inplace=True)
categories.drop_duplicates(subset='id', keep='first', inplace=True)

In [44]:
# merge datasets
df = messages.merge(categories, on='id', how='left')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [45]:
df.shape

(26180, 5)

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [50]:
df['categories'][1]

'related-1;request-0;offer-0;aid_related-1;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-1;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-1;floods-0;storm-1;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0'

In [51]:
cat = df['categories'].str.split(pat=';', expand=True)

In [59]:
colnames = cat.iloc[1,:].str.strip('01-')

In [61]:
cat.columns = colnames

In [86]:
cat['direct_report'].value_counts()

direct_report-0    21116
direct_report-1     5064
Name: direct_report, dtype: int64

In [93]:
cat = cat.replace(r'[a-z_]*-', '', regex=True)

In [101]:
print(df.shape)
print(cat.shape)

(26180, 4)
(26180, 36)


In [100]:
df = df.drop(['categories'], axis=1)

In [110]:
complete_data = df.merge(cat, left_index=True, right_index=True)

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [113]:
engine = create_engine('sqlite:///Fergus.db')
complete_data.to_sql('table_name', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.